In [6]:
import pandas as pd
import sqlite3

In [7]:
orcl_df = pd.read_csv("./ORCL.csv")
msft_df = pd.read_csv("./MSFT.csv")
amzn_df = pd.read_csv("./AMZN.csv")

In [8]:
orcl_df['Ticker'] = "ORCL"
msft_df['Ticker'] = "MSFT"
amzn_df['Ticker'] = "MAZN"

In [9]:
df = pd.concat([orcl_df, msft_df, amzn_df], ignore_index=True)

In [10]:
df = df[['Date','Close','Ticker']]

In [11]:
df.to_csv("./Ticker.csv", index=False)

In [54]:
df.head(4)

,Date,Close,Ticker
0,2021-08-20,88.940002,ORCL
1,2021-08-23,89.120003,ORCL
2,2021-08-24,88.550003,ORCL
3,2021-08-25,88.620003,ORCL


## max close price for each ticker

Calculate the maximum close price for each ticker by using the MAX function in conjunction with a window function

Window function is initiated with the OVER keyword

Specify that the ticker column to partition the data


It will return a table of the same length as the original, but with a new column called MaxPrice that contains the maximum close price for ticker during time period

In [55]:
import sqlite3

# create connection to in memory sqlite db
conn = sqlite3.connect(":memory:")

# save prices dataframe to sqlite db
df.to_sql(name="TickerPriceTable", con=conn, index=False)


query1 = """
    
    SELECT
        date(Date) as Date
        , Ticker
        , Close as ClosingPrice
        , MAX(Close) OVER(PARTITION BY Ticker) as MaxPrice
    FROM
        TickerPriceTable
        
"""

# use pandas read_sql to execute the query and return a dataframe
result1_df = pd.read_sql(query1, con=conn)

In [56]:
result1_df

,Date,Ticker,ClosingPrice,MaxPrice
0,2021-08-20,MAZN,159.997498,184.802994
1,2021-08-23,MAZN,163.293503,184.802994
2,2021-08-24,MAZN,165.289001,184.802994
3,2021-08-25,MAZN,164.959000,184.802994
4,2021-08-26,MAZN,165.800003,184.802994
...,...,...,...,...
751,2022-08-15,ORCL,79.110001,103.650002
752,2022-08-16,ORCL,79.519997,103.650002
753,2022-08-17,ORCL,79.250000,103.650002
754,2022-08-18,ORCL,79.320000,103.650002


## 30 day closing price moving average for each Ticker 

For moving average computation - the order of values within the group are important (It should be in chronological order), 
therefore need to order the values within the group by the date

To define the rolling window, specify that the average should be calculated using the preceding *n* rows & the current row



In [24]:
query2 = """

SELECT
    date(Date) AS Date
    , Ticker
    , Close
    , AVG(Close) OVER(
        PARTITION BY Ticker
        ORDER BY date(Date)
        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
    )
     AS MovingAvg_30_Days
FROM
    TickerPriceTable
    
"""

In [25]:
# use pandas read_sql to execute the query and return a dataframe
result2_df = pd.read_sql(query2, con=conn)

In [26]:
result2_df

,Date,Ticker,Close,MovingAvg_30_Days
0,2021-08-20,MAZN,159.997498,159.997498
1,2021-08-23,MAZN,163.293503,161.645500
2,2021-08-24,MAZN,165.289001,162.860001
3,2021-08-25,MAZN,164.959000,163.384750
4,2021-08-26,MAZN,165.800003,163.867801
...,...,...,...,...
751,2022-08-15,ORCL,79.110001,74.526000
752,2022-08-16,ORCL,79.519997,74.777000
753,2022-08-17,ORCL,79.250000,75.030000
754,2022-08-18,ORCL,79.320000,75.279667


## Get Previous day's Close Price for each Ticker 


In [32]:
query3 = """

SELECT
    date(Date) AS Date
    , Ticker
    , Close
    , LAG(Close, 1) OVER(
        PARTITION BY Ticker
        ORDER BY date(Date)
    ) AS PreviousClose
FROM
    TickerPriceTable
    
"""
 

In [33]:
# use pandas read_sql to execute the query and return a dataframe
result3_df = pd.read_sql(query3, con=conn)

In [34]:
result3_df

,Date,Ticker,Close,PreviousClose
0,2021-08-20,MAZN,159.997498,NaN
1,2021-08-23,MAZN,163.293503,159.997498
2,2021-08-24,MAZN,165.289001,163.293503
3,2021-08-25,MAZN,164.959000,165.289001
4,2021-08-26,MAZN,165.800003,164.959000
...,...,...,...,...
751,2022-08-15,ORCL,79.110001,79.150002
752,2022-08-16,ORCL,79.519997,79.110001
753,2022-08-17,ORCL,79.250000,79.519997
754,2022-08-18,ORCL,79.320000,79.250000


## Daily Percentage Return 

In [42]:
query4 = """

WITH PreviousCloseTickerPrices AS (
SELECT
     date(Date) AS Date,
     Ticker,
     Close,
     LAG(Close, 1) OVER(PARTITION BY Ticker ORDER BY date(Date)) AS PreviousClose
FROM
    TickerPriceTable
)

SELECT
    Date,
    Ticker,
    Close,
    Close/PreviousClose - 1 AS DailyReturn
FROM PreviousCloseTickerPrices

"""

In [43]:
# use pandas read_sql to execute the query and return a dataframe
result4_df = pd.read_sql(query4, con=conn)

In [44]:
result4_df

,Date,Ticker,Close,DailyReturn
0,2021-08-20,MAZN,159.997498,NaN
1,2021-08-23,MAZN,163.293503,0.020600
2,2021-08-24,MAZN,165.289001,0.012220
3,2021-08-25,MAZN,164.959000,-0.001997
4,2021-08-26,MAZN,165.800003,0.005098
...,...,...,...,...
751,2022-08-15,ORCL,79.110001,-0.000505
752,2022-08-16,ORCL,79.519997,0.005183
753,2022-08-17,ORCL,79.250000,-0.003395
754,2022-08-18,ORCL,79.320000,0.000883
